***
@author: jmeier<br>
date: 08. january 2024<br>
short description: 
-  Programm uses google travel times from database to generate a map showing the closest clinic from any location listed<br>
-  Generates two maps. One for travel by car and one for travel by transit.
***

In [1]:
import sqlite3
import pandas as pd
import geopandas as gpd
pd.options.mode.chained_assignment = None  # default='warn'  --> catches unnecessary pandas warning

In [2]:
conn = sqlite3.connect("../communities/communities.sqlite")
cur = conn.cursor()

In [3]:
# Reads in .shp (shape) file with the shapes of all federal states of germany. Selects bavaria
states = gpd.read_file("../shapefile/DEU_adm1.shp")
bavaria = states.loc[states["NAME_1"] == "Bayern"]
bavaria

,ID_0,ISO,NAME_0,ID_1,NAME_1,TYPE_1,ENGTYPE_1,NL_NAME_1,VARNAME_1,geometry
1,86,DEU,Germany,2,Bayern,Land,State,None,Bavaria,"POLYGON ((10.13386 50.55000, 10.13980 50.54252..."


In [4]:
# Generates two maps (one for travel by car and one by transit) highlighting the shape of bavaria --> Travel times are addes later
overview_map = bavaria.explore(tooltip=False, highlight=False, style_kwds={"color": "#434343", "fill": False})
overview_map_transit = bavaria.explore(tooltip=False, highlight=False, style_kwds={"color": "#434343", "fill": False})
overview_map

In [5]:
# Generates a pandas DataFrame containing the locations of al clinica, transforms it to a GeoPandas GeoDataFrame and maps (with marker symbols) it on the two existing maps 
clinica_map = pd.DataFrame({
                        "id":   ["TUM",     "LMU",     "Würzburg", "Erlangen", "Regensburg", "Augsburg"],
                        "lat":  [48.135833, 48.111388, 49.800833,  49.599444,  48.987778,    48.385833], #Breitengrad Latitude y
                        "lon":  [11.599167, 11.469444,  9.953611,  11.010556,  12.090278,    10.837500]  #Längengrad Longitude x
                        
})

clinica_map_geo = gpd.GeoDataFrame(clinica_map,
                                   geometry=gpd.points_from_xy(clinica_map.lon, clinica_map.lat),
                                   crs="EPSG:4326"
                                   )

overview_map = clinica_map_geo.explore(m=overview_map, marker_type="marker")
overview_map_transit = clinica_map_geo.explore(m=overview_map_transit, marker_type="marker")
overview_map

In [6]:
# Selects communities including google travel distances and times from database where google_retrieved = 1 and Land.id = 9 (Bayern). Saves it as DataFrame
cur.execute("""SELECT   Gemeinde.id, 
                        Gemeinde.name, 
                        Gemeinde.breitengrad, 
                        Gemeinde.längengrad, 
                        Gemeinde.bevölkerung_insgesamt ,
                        google_thur7am_car_distance_lmu,
                        google_thur7am_car_time_lmu,
                        google_thur7am_car_distance_tum,
                        google_thur7am_car_time_tum,
                        google_thur7am_car_distance_würzburg,
                        google_thur7am_car_time_würzburg,
                        google_thur7am_car_distance_erlangen,
                        google_thur7am_car_time_erlangen,
                        google_thur7am_car_distance_regensburg,
                        google_thur7am_car_time_regensburg,
                        google_thur7am_car_distance_augsburg,
                        google_thur7am_car_time_augsburg,
                        google_thur7am_transit_distance_lmu,
                        google_thur7am_transit_time_lmu,
                        google_thur7am_transit_distance_tum,
                        google_thur7am_transit_time_tum,
                        google_thur7am_transit_distance_würzburg,
                        google_thur7am_transit_time_würzburg,
                        google_thur7am_transit_distance_erlangen,
                        google_thur7am_transit_time_erlangen,
                        google_thur7am_transit_distance_regensburg,
                        google_thur7am_transit_time_regensburg,
                        google_thur7am_transit_distance_augsburg,
                        google_thur7am_transit_time_augsburg
            FROM Gemeinde JOIN VB JOIN Kreis JOIN RB JOIN Land 
                ON  Gemeinde.vb_id  = VB.id
                AND VB.kreis_id     = Kreis.id
                AND Kreis.rb_id     = RB.id
                AND RB.land_id      = Land.id
            WHERE Gemeinde.google_retrieved = 1 AND Land.id = 9""")
communities = pd.DataFrame(cur.fetchall(), columns=["Gemeinde.id", 
                                                    "Gemeinde.name", 
                                                    "Gemeinde.breitengrad", 
                                                    "Gemeinde.längengrad", 
                                                    "Gemeinde.bevölkerung_insgesamt",
                                                    "google_thur7am_car_distance_lmu",
                                                    "google_thur7am_car_time_lmu",
                                                    "google_thur7am_car_distance_tum",
                                                    "google_thur7am_car_time_tum",
                                                    "google_thur7am_car_distance_würzburg",
                                                    "google_thur7am_car_time_würzburg",
                                                    "google_thur7am_car_distance_erlangen",
                                                    "google_thur7am_car_time_erlangen",
                                                    "google_thur7am_car_distance_regensburg",
                                                    "google_thur7am_car_time_regensburg",
                                                    "google_thur7am_car_distance_augsburg",
                                                    "google_thur7am_car_time_augsburg",
                                                    "google_thur7am_transit_distance_lmu",
                                                    "google_thur7am_transit_time_lmu",
                                                    "google_thur7am_transit_distance_tum",
                                                    "google_thur7am_transit_time_tum",
                                                    "google_thur7am_transit_distance_würzburg",
                                                    "google_thur7am_transit_time_würzburg",
                                                    "google_thur7am_transit_distance_erlangen",
                                                    "google_thur7am_transit_time_erlangen",
                                                    "google_thur7am_transit_distance_regensburg",
                                                    "google_thur7am_transit_time_regensburg",
                                                    "google_thur7am_transit_distance_augsburg",
                                                    "google_thur7am_transit_time_augsburg"
                                                    ])




communities.head()

,Gemeinde.id,Gemeinde.name,Gemeinde.breitengrad,Gemeinde.längengrad,Gemeinde.bevölkerung_insgesamt,google_thur7am_car_distance_lmu,google_thur7am_car_time_lmu,google_thur7am_car_distance_tum,google_thur7am_car_time_tum,google_thur7am_car_distance_würzburg,...,google_thur7am_transit_distance_tum,google_thur7am_transit_time_tum,google_thur7am_transit_distance_würzburg,google_thur7am_transit_time_würzburg,google_thur7am_transit_distance_erlangen,google_thur7am_transit_time_erlangen,google_thur7am_transit_distance_regensburg,google_thur7am_transit_time_regensburg,google_thur7am_transit_distance_augsburg,google_thur7am_transit_time_augsburg
0,6300,Ingolstadt,48.765420,11.423387,141029,96433,4083,80937,3554,201799,...,88578.0,4992.0,225991.0,7776.0,111812.0,6737.0,83065.0,6535.0,77900.0,7931.0
1,6301,"München, Landeshauptstadt",48.137683,11.575997,1512491,12289,1739,2607,494,276237,...,2404.0,851.0,303180.0,9439.0,224063.0,7275.0,145074.0,7994.0,68635.0,4723.0
2,6302,Rosenheim,47.856358,12.128932,64403,75875,3798,68037,3364,347018,...,57307.0,3089.0,362338.0,12413.0,283221.0,10249.0,204251.0,10848.0,127617.0,8117.0
3,6303,"Altötting, St",48.227986,12.678920,13104,117893,5095,91684,3725,357043,...,90949.0,6300.0,395914.0,15624.0,317148.0,13805.0,137521.0,9739.0,161163.0,10608.0
4,6304,"Burghausen, St",48.168825,12.831282,19364,134741,5631,108532,4262,373891,...,108840.0,7280.0,413805.0,16604.0,335039.0,14785.0,155412.0,10719.0,179054.0,11588.0


In [7]:
# Makes two new columns in the communities DataFrame where the minimal travel times by car and transit are saved
communities["google_min_time_car"] = communities[["google_thur7am_car_time_lmu", 
                                                  "google_thur7am_car_time_tum",
                                                  "google_thur7am_car_time_würzburg",
                                                  "google_thur7am_car_time_erlangen",
                                                  "google_thur7am_car_time_regensburg",
                                                  "google_thur7am_car_time_augsburg"]].min(axis=1)

communities["google_min_time_transit"] = communities[["google_thur7am_transit_time_lmu", 
                                                  "google_thur7am_transit_time_tum",
                                                  "google_thur7am_transit_time_würzburg",
                                                  "google_thur7am_transit_time_erlangen",
                                                  "google_thur7am_transit_time_regensburg",
                                                  "google_thur7am_transit_time_augsburg"]].min(axis=1)

# Additionally the minimal times are transformed from seconds to hours and saved in separate columns in the communities DataFrame
communities["car_time_h"] = round(communities["google_min_time_car"] / 3600 , 1)
communities["transit_time_h"] = round(communities["google_min_time_transit"] / 3600, 1)

communities["Nearest clinic"] = None # new DataFrame column --> here nearest clinic will be added

communities.head()

,Gemeinde.id,Gemeinde.name,Gemeinde.breitengrad,Gemeinde.längengrad,Gemeinde.bevölkerung_insgesamt,google_thur7am_car_distance_lmu,google_thur7am_car_time_lmu,google_thur7am_car_distance_tum,google_thur7am_car_time_tum,google_thur7am_car_distance_würzburg,...,google_thur7am_transit_time_erlangen,google_thur7am_transit_distance_regensburg,google_thur7am_transit_time_regensburg,google_thur7am_transit_distance_augsburg,google_thur7am_transit_time_augsburg,google_min_time_car,google_min_time_transit,car_time_h,transit_time_h,Nearest clinic
0,6300,Ingolstadt,48.765420,11.423387,141029,96433,4083,80937,3554,201799,...,6737.0,83065.0,6535.0,77900.0,7931.0,3554,4992.0,1.0,1.4,None
1,6301,"München, Landeshauptstadt",48.137683,11.575997,1512491,12289,1739,2607,494,276237,...,7275.0,145074.0,7994.0,68635.0,4723.0,494,851.0,0.1,0.2,None
2,6302,Rosenheim,47.856358,12.128932,64403,75875,3798,68037,3364,347018,...,10249.0,204251.0,10848.0,127617.0,8117.0,3364,3089.0,0.9,0.9,None
3,6303,"Altötting, St",48.227986,12.678920,13104,117893,5095,91684,3725,357043,...,13805.0,137521.0,9739.0,161163.0,10608.0,3725,6300.0,1.0,1.8,None
4,6304,"Burghausen, St",48.168825,12.831282,19364,134741,5631,108532,4262,373891,...,14785.0,155412.0,10719.0,179054.0,11588.0,4262,7280.0,1.2,2.0,None


In [8]:
# Transforms the communities DataFrame to a GeoDataFrame
communities_geo = gpd.GeoDataFrame (
                    communities, 
                    geometry=gpd.points_from_xy(communities["Gemeinde.längengrad"], communities["Gemeinde.breitengrad"]), 
                    crs="EPSG:4326"
                    )

communities_geo.head()

,Gemeinde.id,Gemeinde.name,Gemeinde.breitengrad,Gemeinde.längengrad,Gemeinde.bevölkerung_insgesamt,google_thur7am_car_distance_lmu,google_thur7am_car_time_lmu,google_thur7am_car_distance_tum,google_thur7am_car_time_tum,google_thur7am_car_distance_würzburg,...,google_thur7am_transit_distance_regensburg,google_thur7am_transit_time_regensburg,google_thur7am_transit_distance_augsburg,google_thur7am_transit_time_augsburg,google_min_time_car,google_min_time_transit,car_time_h,transit_time_h,Nearest clinic,geometry
0,6300,Ingolstadt,48.765420,11.423387,141029,96433,4083,80937,3554,201799,...,83065.0,6535.0,77900.0,7931.0,3554,4992.0,1.0,1.4,None,POINT (11.42339 48.76542)
1,6301,"München, Landeshauptstadt",48.137683,11.575997,1512491,12289,1739,2607,494,276237,...,145074.0,7994.0,68635.0,4723.0,494,851.0,0.1,0.2,None,POINT (11.57600 48.13768)
2,6302,Rosenheim,47.856358,12.128932,64403,75875,3798,68037,3364,347018,...,204251.0,10848.0,127617.0,8117.0,3364,3089.0,0.9,0.9,None,POINT (12.12893 47.85636)
3,6303,"Altötting, St",48.227986,12.678920,13104,117893,5095,91684,3725,357043,...,137521.0,9739.0,161163.0,10608.0,3725,6300.0,1.0,1.8,None,POINT (12.67892 48.22799)
4,6304,"Burghausen, St",48.168825,12.831282,19364,134741,5631,108532,4262,373891,...,155412.0,10719.0,179054.0,11588.0,4262,7280.0,1.2,2.0,None,POINT (12.83128 48.16882)


In [9]:
# Parameters for the map. Color and clinic_list are for marker color on map and tooltip clinic name
colors =        ["blue",        "red",          "green",        "orange",       "grey",         "black"]
clinic_list =   ["München-LMU", "München-TUM",  "Würzburg",     "Erlangen",     "Regensburg",   "Augsburg"]

# car_times and transit_time lists are to select correct entry from Dataframe. Same order as for clinic_list.
car_times = [ 
        "google_thur7am_car_time_lmu",
        "google_thur7am_car_time_tum",
        "google_thur7am_car_time_würzburg",
        "google_thur7am_car_time_erlangen",
        "google_thur7am_car_time_regensburg",
        "google_thur7am_car_time_augsburg"
        ]

transit_times = [ 
        "google_thur7am_transit_time_lmu",
        "google_thur7am_transit_time_tum",
        "google_thur7am_transit_time_würzburg",
        "google_thur7am_transit_time_erlangen",
        "google_thur7am_transit_time_regensburg",
        "google_thur7am_transit_time_augsburg"
        ]

In [10]:
# Generates car travel map (saves it as html)
# Loop over all clinic
for clinic in range(6):    

    # Selects entries where the shortest time (google_min_time_car) ist to the selected clinic car_times[clinic]. 
    # Sets the clinic name in column "Nearest clinic" of selected DataFrame
    # Drops all rows with at least one na value in the car travel times 
    clinic_geodf_sel = communities_geo.loc[(communities_geo[car_times[clinic]] == communities_geo["google_min_time_car"])]
    clinic_geodf_sel["Nearest clinic"] = clinic_list[clinic]
    clinic_geodf_sel.dropna(axis=0, inplace=True, subset=car_times)        

    # Generates circle markers on community location in color of the closest clinic. Size of the cirlce depends on travel time.
    clinic_geodf_b30 = clinic_geodf_sel.loc[(clinic_geodf_sel["google_min_time_car"] < 1800)]
    if len(clinic_geodf_b30) > 0:
        overview_map = clinic_geodf_b30.explore(m=overview_map, marker_type="circle_marker", marker_kwds={"radius":5}, color=colors[clinic], tooltip=["Gemeinde.name", "Gemeinde.bevölkerung_insgesamt", "car_time_h", "Nearest clinic"])
    
    clinic_geodf_b60 = clinic_geodf_sel.loc[(clinic_geodf_sel["google_min_time_car"] >= 1800) & (clinic_geodf_sel["google_min_time_car"] < 3600)]
    if len(clinic_geodf_b60) > 0:
        overview_map = clinic_geodf_b60.explore(m=overview_map, marker_type="circle_marker", marker_kwds={"radius":10}, color=colors[clinic], tooltip=["Gemeinde.name", "Gemeinde.bevölkerung_insgesamt", "car_time_h", "Nearest clinic"])
    
    clinic_geodf_b120 = clinic_geodf_sel.loc[(clinic_geodf_sel["google_min_time_car"] >= 3600) & (clinic_geodf_sel["google_min_time_car"] < 7200)]
    if len(clinic_geodf_b120) > 0:
        overview_map = clinic_geodf_b120.explore(m=overview_map, marker_type="circle_marker", marker_kwds={"radius":20}, color=colors[clinic], tooltip=["Gemeinde.name", "Gemeinde.bevölkerung_insgesamt", "car_time_h", "Nearest clinic"])
    
    clinic_geodf_a120 = clinic_geodf_sel.loc[(clinic_geodf_sel["google_min_time_car"] >= 7200)]
    if len(clinic_geodf_a120) > 0:
        overview_map = clinic_geodf_a120.explore(m=overview_map, marker_type="circle_marker", marker_kwds={"radius":30}, color=colors[clinic], tooltip=["Gemeinde.name", "Gemeinde.bevölkerung_insgesamt", "car_time_h", "Nearest clinic"])


overview_map.save("car_thur7am.html")
overview_map


In [11]:
# Generates transit travel map (saves it as html)
# Loop over all clinic
for clinic in range(6):
    
    # Selects entries where the shortest time (google_min_time_car) ist to the selected clinic car_times[clinic]. 
    # Sets the clinic name in column "Nearest clinic" of selected DataFrame
    # Drops all rows with at least one na value in the transit travel times    
    clinic_geodf_sel = communities_geo.loc[(communities_geo[transit_times[clinic]] == communities_geo["google_min_time_transit"])]
    clinic_geodf_sel["Nearest clinic"] = clinic_list[clinic]
    clinic_geodf_sel.dropna(axis=0, inplace=True, subset=transit_times)    

    # Generates circle markers on community location in color of the closest clinic. Size of the cirlce depends on travel time.
    clinic_geodf_b30 = clinic_geodf_sel.loc[(clinic_geodf_sel["google_min_time_transit"] < 1800)]
    if len(clinic_geodf_b30) > 0:
        overview_map_transit = clinic_geodf_b30.explore(m=overview_map_transit, marker_type="circle_marker", marker_kwds={"radius":5}, color=colors[clinic], tooltip=["Gemeinde.name", "Gemeinde.bevölkerung_insgesamt", "transit_time_h", "Nearest clinic"])
    
    clinic_geodf_b60 = clinic_geodf_sel.loc[(clinic_geodf_sel["google_min_time_transit"] >= 1800) & (clinic_geodf_sel["google_min_time_transit"] < 3600)]
    if len(clinic_geodf_b60) > 0:
        overview_map_transit = clinic_geodf_b60.explore(m=overview_map_transit, marker_type="circle_marker", marker_kwds={"radius":10}, color=colors[clinic], tooltip=["Gemeinde.name", "Gemeinde.bevölkerung_insgesamt", "transit_time_h", "Nearest clinic"])
    
    clinic_geodf_b120 = clinic_geodf_sel.loc[(clinic_geodf_sel["google_min_time_transit"] >= 3600) & (clinic_geodf_sel["google_min_time_transit"] < 7200)]
    if len(clinic_geodf_b120) > 0:
        overview_map_transit = clinic_geodf_b120.explore(m=overview_map_transit, marker_type="circle_marker", marker_kwds={"radius":20}, color=colors[clinic], tooltip=["Gemeinde.name", "Gemeinde.bevölkerung_insgesamt", "transit_time_h", "Nearest clinic"])
    
    clinic_geodf_a120 = clinic_geodf_sel.loc[(clinic_geodf_sel["google_min_time_transit"] >= 7200)]
    if len(clinic_geodf_a120) > 0:
        overview_map_transit = clinic_geodf_a120.explore(m=overview_map_transit, marker_type="circle_marker", marker_kwds={"radius":30}, color=colors[clinic], tooltip=["Gemeinde.name", "Gemeinde.bevölkerung_insgesamt", "transit_time_h", "Nearest clinic"])

overview_map_transit.save("transit_thur7am.html")
overview_map_transit  
